In [1]:
import numpy as np
from ultralytics import YOLO
import cv2
import os
import csv
import pickle

FRAMES_PER_SEQUENCE = 6
OVERLAP = 3

rawvideos_dir = "raw"
video_fp = "raw/short10.mp4" #video path
output_dir = "labelled" 
output_name = f"{video_fp[len(rawvideos_dir)+1:][:-4]}"
frames_fp = f"{output_dir}/{output_name}/frames"

In [31]:
'''
Params: 
frame_path: frames 
boxes_path: csv file with
    - Column 0: video name
    - Column 1: sequence number
    - Column 2: boxes number 
    - Column 3: 
'''

def label_person_ids(boxes_path):
    data = np.genfromtxt(boxes_path, delimiter=',')[:,1:]
    video_name = np.genfromtxt(boxes_path, dtype=str, delimiter=',')[0,0]
    sequences = np.unique(data[:,0])
    
    # Make a directory for sequence data if not already exist
    sequence_path = f'labelled/{video_name}/sequences'
    if not os.path.exists(sequence_path):
        os.makedirs(sequence_path)
    for seq in sequences:
        seq = int(seq)
        sequence_data = data[data[:,0]==seq,:]
        
        beg_frame, end_frame = int(sequence_data[0,1]), int(sequence_data[-1,1])
        for frame_num in range(beg_frame, end_frame+1):
            # find the frame image to edit
            frame_name = f'labelled/{video_name}/frames/frame_{frame_num:04d}'
            frame_img = cv2.imread(f'{frame_name}.jpg')

            # label the person ID on the bounding box
            frame_data = sequence_data[sequence_data[:,1]==frame_num,:] 
            for i in range(frame_data.shape[0]):
                x1,y1 = frame_data[i,3],frame_data[i,4]
                person_ID = int(frame_data[i,2])
                cv2.putText(frame_img, str(person_ID), (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 3)
            
            cv2.imwrite(f'{sequence_path}/sequence_{seq:04d}_frame_{frame_num-beg_frame}.jpg', frame_img)



In [35]:
label_person_ids(f'{output_dir}/{output_name}/boxes.csv')